In [20]:
from src.datasets import Ters_dataset_filtered_skip
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

In [24]:
data_path = "/scratch/phys/sin/sethih1/data_files/planar_molecules_256"
num_channels = 400
sg_ch = True

ters_set = Ters_dataset_filtered_skip(
        filename=data_path,
        frequency_range=[0, 4000],  # adjust as needed
        num_channels=num_channels,
        std_deviation_multiplier=2,
        sg_ch=sg_ch,
        t_image=None,
        t_freq=None
    )

ters_loader = DataLoader(
    ters_set,
    batch_size=32,
    shuffle=False,
)

In [25]:
device = 'cpu'
model = torch.load('/scratch/phys/sin/sethih1/models/planar_256/config7/seg_bs_32_lr_0.0001_loss_dice_loss.pt', map_location=device)
model.eval()

/tmp/ipykernel_3137151/2864357902.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/scratch/phys/sin/sethih1/models/planar_256/config7/seg_bs_32_lr_0.

AttentionUNet(
  (conv): Conv2d(400, 64, kernel_size=(1, 1), stride=(1, 1))
  (encoder): ModuleList(
    (0): ResBlock(
      (res): Sequential(
        (0): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
      (skip): Sequential(
        (0): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ResBlock(
      (res): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [ ]:
def batch_iou(pred_logits, true_masks, num_classes, threshold=0.5):
    """
    pred_logits: torch.Tensor, shape (B, C, H, W) for C>1, or (B,1,H,W) for binary
    true_masks:  torch.Tensor, shape (B, H, W) with integer labels 0..C-1 (binary: 0/1)
    returns:     list of intersections, list of unions (each length=num_classes)
    """
    B = true_masks.size(0)
    device = true_masks.device

    # 3. Post‐process 
    if pred_logits.shape[1] > 1:
        # multi‐class → take argmax
        preds = torch.argmax(pred_logits, dim=1)
    else:
        # binary → threshold the single‐channel sigmoid/logit output
        probs = torch.sigmoid(pred_logits) if pred_logits.dtype != torch.bool else pred_logits
        preds = (probs > threshold).long().squeeze(1)

    # initialize accumulators
    total_inter = torch.zeros(num_classes, device=device)
    total_union = torch.zeros(num_classes, device=device)

    # 4. for each class, compute intersection & union
    for cls in range(num_classes):
        pred_inds = (preds == cls)
        true_inds = (true_masks == cls)

        # intersection & union
        inter = (pred_inds & true_inds).sum(dim=(1,2))  # per‐image
        union = (pred_inds | true_inds).sum(dim=(1,2))

        # sum over the batch
        total_inter[cls] += inter.sum()
        total_union[cls] += union.sum()

    return total_inter, total_union

# 5. Loop over dataset and accumulate
num_classes = 2  # ← change to your number of classes
running_inter = torch.zeros(num_classes, device=device)
running_union = torch.zeros(num_classes, device=device)

with torch.no_grad():
    for batch in tqdm(ters_loader, desc="Evaluating IoU"):
        # unpack your batch; adjust if your Dataset returns a dict
        images,_, masks = batch  
        images = images.to(device)
        masks  = masks.to(device)

        outputs = model(images)                # (B, C, H, W) or (B,1,H,W)
        inters, unions = batch_iou(outputs, masks, num_classes)

        running_inter += inters
        running_union += unions

# 6. Compute per‐class and mean IoU
per_class_iou = running_inter / (running_union + 1e-6)
mean_iou      = per_class_iou.mean().item()

print(f"Per-class IoU: {per_class_iou.tolist()}")
print(f"Mean IoU over {num_classes} classes: {mean_iou:.4f}")

Evaluating IoU:   0%|          | 0/41 [00:00<?, ?it/s]